In [1]:
from scraping_utils import *

csv_data/                    __init__.py*         scraping_utils.pyc
eppraisal_sold_homes.py      load_properties.py*  soldhomedata_20180207.csv
image_filter_and_upload.py*  __pycache__/         sold_homes_scrape.py*
image_utils.py*              remax_scrape.py*     Untitled.ipynb
image_utils.pyc              scraping_utils.py*


In [5]:
from bs4 import BeautifulSoup
from requests import get

In [6]:
url = 'https://www.eppraisal.com/recently-sold/nc/charlotte/?pg=1'

In [7]:
eppraisalsoup = BeautifulSoup(get(url).text,'html.parser')

In [8]:
eppraisalsoup

<!DOCTYPE doctype html>

<html>
<head>
<meta charset="utf-8">
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<meta content="Research recent property sales in and around Charlotte, NC. | page 1" name="description" property="description"/>
<meta content="LgvqqTzYl176PCus2QoK3q1yPpxPrOaUhNRbu+fMwbo=" name="verify-v1"/>
<meta content="Eppraisal" name="author"/>
<meta content="Copyright (c) 2018 eppraisal.com." name="Copyright"/>
<meta content="Eppraisal" property="og:site_name"/>
<meta content="company" property="og:type"/>
<title>Recently Sold Homes in Charlotte, NC | page 1</title>
<link href="/img/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="//fonts.googleapis.com/css?family=Lato:300" rel="stylesheet" type="text/css">
<link href="/css/reset.css" rel="stylesheet" type="text/css">
<link href="/css/fonts.css" rel="stylesheet" type="text/css">
<link href="/css/style.css" rel="stylesheet" type="text/css">
<link href="https://www.eppraisal.com/rec

In [20]:
def get_property_urls_from_eplp(eppraisal_url):
    """This pulls a list of urls from the eppraisal site
    
    """
    BASE_URL = 'https://www.eppraisal.com'
    eppraisal_soup = BeautifulSoup(get(eppraisal_url).text,'html.parser')
    property_urls = []
    for home in eppraisal_soup.find_all('a',class_='proplink'):
        home_url_addon = home['href']
        home_url = BASE_URL + home_url_addon
        property_urls.append(home_url)
    return property_urls
        
    

In [21]:
eppraisalsoup.find_all('a',class_='proplink')[0]['href']

'/home-values/property-lookup?m=16833greenlawnhillsct28213'

In [29]:
def get_property_soup(property_url):
    """returns a bs4 soup object from a url """
    soup = BeautifulSoup(get(property_url).text,'html.parser')
    return soup


In [24]:
urls = get_property_urls_from_eplp(url)

In [27]:
prop = get_property_soup(urls[0])

In [87]:
def scrape_eppraisal_home_page(home_url):
    home = get_property_soup(home_url)
    
    

In [264]:
def find_sale_history_from_soup(soup):
    """Extracts the sale history from bs4 soup object
    
    Keyword arguments:
        soup: [bs4 soup object] soup obj of property to scrape
    
    Returns:
        sale_hist: [json] json object of dict of sale history
    
    """
    sales_hist_loop = soup.find_all('div',class_='panel-column panel-column-3')
    sales_hist_dict = {}
    for idx, item in enumerate(sales_hist_loop):
        if 'Last Sold' in item.find_all('li')[0].text:
            datestr = item.find_all('li')[0].text
            date = str(datestr[datestr.find('Last Sold: ')+len('Last Sold: '):])
            parsed_date = parser.parse(date).strftime('%Y%m%d')
            salestr = sales_hist_loop[idx+1].text
            sale = salestr[salestr.find('Sold Amt: $')+len('Sold Amt: $'):]
            sale = int(sale.replace(',',''))
            sales_hist_dict[parsed_date] = sale
    sales_hist_json = json.dumps(sales_hist_dict)
    return sales_hist_json
    
    

In [228]:
from dateutil import parser
import json


sales_hist_loop = prop.find_all('div',class_='panel-column panel-column-3')
sales_hist_dict = {}
for idx, item in enumerate(sales_hist_loop):
    if 'Last Sold' in item.find_all('li')[0].text:
        datestr = item.find_all('li')[0].text
        date = str(datestr[datestr.find('Last Sold: ')+len('Last Sold: '):])
        parsed_date = parser.parse(date,dayfirst=True).strftime('%Y%m%d')
        salestr = sales_hist_loop[idx+1].text
        sale = salestr[salestr.find('Sold Amt: $')+len('Sold Amt: $'):]
        sale = int(sale.replace(',',''))
        sales_hist_dict[parsed_date] = sale
    
        

In [229]:
find_sale_history_from_soup(prop)

'{"20170802": 108000, "20070702": 107000}'

In [109]:
def get_address_from_soup(soup,city,state):
    """Pulls the address from the soup, checks with house canary API
    
    Keyword arguments:
        soup: [bs4 soup obj] soup object of property to scrape
        city: [string] city name of property e.g. Charlotte
        state: [string] state (2 name abbr) of property e.g. NC
    
    """
    addr_string = soup.find_all('h1',class_='hero-title')[0].text.replace('»','').strip()
    address_line_1 = addr[:addr.find(city)].strip()
    state = state
    city = city
    unit = ''
    zipcode = addr[addr.find(state)+len(state):].strip()
    scrape_address = {
        'address_line1': address_line_1,
        'city': city,
        'state': state,
        'zipcode': zipcode,
        'unit': unit, 
    }
    response = canonicalizeAddress(scrape_address)
    return response
    

In [110]:
def canonicalizeAddress(remax_address_dict):
    """
    this takes a dictionary of address data scraped from the remax website and calls the housecanary API
    to make sure that all of the data is standardized
    
    input
    
    remax_address_dict [dict] {
        address_line1: [string] steet number, street
        unit: [int] (optional) unit no
        state: [string] state, two letter abbreviation
        zipcode: [int] - five number zipcode
    }
    
    returns [dict] of street address info from the house canary API 
    
    """



    hc_key = '9SJCA9DISJVUAVAS4QQQ'
    hc_secret = '80vGOq9qYEy46a53XsUReFKpyvPK1owG' 
    if 'unit' in remax_address_dict:
        params = {
        
        'address': remax_address_dict['address_line1'],
        'state': remax_address_dict['state'],
        'zipcode': remax_address_dict['zipcode'],
        'city': remax_address_dict['city'],
        'unit':remax_address_dict['unit']
        }
    else:

        params = {
            
            'address': remax_address_dict['address_line1'],
            'state': remax_address_dict['state'],
            'zipcode': remax_address_dict['zipcode'],
            'city': remax_address_dict['city'],
        }
    geocode_url = 'https://api.housecanary.com/v2/property/geocode'
    response = requests.get(geocode_url, params=params, auth=(hc_key, hc_secret))
    response = response.json()
    return response

In [94]:
addr = prop.find_all('h1',class_='hero-title')[0].text.replace('»','').strip()

In [98]:
addr[:addr.find('Charlotte')].strip()

'16833 Greenlawn Hills Ct'

In [103]:
addr[addr.find('NC')+len('NC'):].strip()

'28213'

In [113]:
get_address_from_soup(prop,'Charlotte','NC')

[{'address_info': {'address': '16833 Greenlawn Hills Ct',
   'address_full': '16833 Greenlawn Hills Ct Charlotte NC 28213',
   'block_id': '371190056143006',
   'blockgroup_id': '371190056143',
   'city': 'Charlotte',
   'county_fips': '37119',
   'geo_precision': 'rooftop',
   'lat': 35.3034,
   'lng': -80.69021,
   'metrodiv': None,
   'msa': '16740',
   'slug': '16833-Greenlawn-Hills-Ct-Charlotte-NC-28213',
   'state': 'NC',
   'status': {'changes': [],
    'details': ['Address fully verified'],
    'errors': [],
    'match': True,
    'requested_item': {'address': '16833 Greenlawn Hills Ct',
     'city': 'Charlotte',
     'state': 'NC',
     'unit': '',
     'zipcode': '28213'}},
   'unit': None,
   'zipcode': '28213',
   'zipcode_plus4': '5985'},
  'property/geocode': {'api_code': 0,
   'api_code_description': 'ok',
   'result': True}}]

In [194]:
def get_bed_bath_sqft_from_soup(soup):
    """This pulls the number of beds, baths, and sq footage from prop
    
    Keyword arguments:
        soup: [bs4 soup object] soup obj of property
        
    Returns:
        listing_dict: dict with beds, baths, sq footage
    
    """
    loop = prop.find_all('div',class_='panel-column panel-column-1 propdesc')[0].find_all('li')
    listing_data = {}
    for line in loop:
        if 'Beds' in line.text:
            beds = int(line.text[line.text.find('Beds: ')+len('Beds: '):])
            listing_data['num_bedrooms'] = beds
        if 'Baths' in line.text:
            baths = int(line.text[line.text.find('Baths: ')+len('Baths: '):])
            listing_data['num_bathrooms'] = baths
        if 'Sqft' in line.text:
            sqft = int(line.text[line.text.find('Sqft: ')+len('Sqft: '):])
            listing_data['building_area_sq_ft'] = sqft
        if 'Lot Area (sq ft): ' in line.text:
            lot_area_sqft = line.text[line.text.find('Lot Area (sq ft): ')+len('Lot Area (sq ft): '):]
            listing_data['lot_area_sqft'] = lot_area_sqft
        if 'Acres: ' in line.text:
            lot_area_acres = float(line.text[line.text.find('Acres: ')+len('Acres: '):])
            listing_data['lot_area_acres'] = lot_area_acres
        if 'Fireplace: ' in line.text:
            fireplace = line.text[line.text.find('Fireplace: ')+len('Fireplace: '):]
            listing_data['fireplace' ] = fireplace
        if 'Heat Type: ' in line.text:
            heat_type = line.text[line.text.find('Heat Type: ')+len('Heat Type: '):]
            listing_data['heat_type' ] = heat_type
        if 'Roof Type: ' in line.text:
            roof_type = line.text[line.text.find('Roof Type: ')+len('Roof Type: '):]
            listing_data['roof_type' ] = fireplace
        if 'Garage/Park sqft: ' in line.text:
            garage_or_park_sqft = line.text[line.text.find('Garage/Park sqft: ')+len('Garage/Park sqft: '):]
            listing_data['garage_or_park_sqft' ] = garage_or_park_sqft
        if 'Basement Area: ' in line.text:
            basement_area = line.text[line.text.find('Basement Area: ')+len('Basement Area: '):]
            listing_data['basement_area' ] = basement_area
        if 'Air Cond: ' in line.text:
            air_conditioning = line.text[line.text.find('Air Cond: ')+len('Air Cond: '):]
            listing_data['air_conditioning' ] = air_conditioning
    return listing_data

In [195]:
loop = prop.find_all('div',class_='panel-column panel-column-1 propdesc')[0].find_all('li')


In [196]:
'Beds' in loop[0].text

True

In [197]:
loop

[<li><strong>Beds:</strong> 2</li>,
 <li><strong>Baths:</strong> 2</li>,
 <li><strong>Sqft:</strong> 1094</li>,
 <li class="prop_more"><strong>APN/Parcel:</strong> 05144907</li>,
 <li class="prop_more"><strong>Lot Area (sq ft):</strong> -</li>,
 <li class="prop_more"><strong>Acres:</strong> 0.0</li>,
 <li class="prop_more"><strong>Fireplace:</strong> </li>,
 <li class="prop_more"><strong>Basement Area:</strong> </li>,
 <li class="prop_more"><strong>Heat Type:</strong> Yes</li>,
 <li class="prop_more"><strong>Roof Type:</strong> </li>,
 <li class="prop_more"><strong>Garage/Park sqft:</strong> </li>,
 <li class="prop_more"><strong>Air Cond:</strong> Central</li>]

In [198]:
get_bed_bath_sqft_from_soup(prop)

{'air_conditioning': 'Central',
 'basement_area': '',
 'building_area_sq_ft': 1094,
 'fireplace': '',
 'garage_or_park_sqft': '',
 'heat_type': 'Yes',
 'lot_area_acres': 0.0,
 'lot_area_sqft': '-',
 'num_bathrooms': 2,
 'num_bedrooms': 2,
 'roof_type': ''}

In [218]:
def get_year_built_from_soup(soup):
    """Pulls the year the home was built from the bs4 soup obj
    """
    sents = soup.find_all('p',class_="main-page-description")[0].text.split('.')
    year_list = list(range(1900,datetime.now().year))
    year_list = [str(x) for x in year_list]
    for sent in sents:
        if 'Property records' in sent:
            tokens = sent.split(' ')
            for word in tokens:
                if word in year_list:
                    year_built = word
                    return year_built
    return None

In [203]:
sents = prop.find_all('p',class_="main-page-description")[0].text.split('.')

In [217]:
year_list = list(range(1900,datetime.now().year))
year_list = [str(x) for x in year_list]
for sent in sents:
    if 'Property records' in sent:
        tokens = sent.split(' ')
        for word in tokens:
            if word in year_list:
                print(word)

2007


In [232]:
get_year_built_from_soup(prop)

'2007'

In [233]:
json_yr = find_sale_history_from_soup(prop)

In [258]:
def get_last_sold_from_history(sale_hist_json):
    """Reads the sale history json and returns the last sell date
    """
    d = json.loads(sale_hist_json)
    d2 = {parser.parse(k): v for k, v in d.items()}
    last_sold_date, last_sold_px = sorted(d2.items(),key=lambda x:x,reverse=True)[0]
    return last_sold_date.strftime('%Y-%m-%d'), last_sold_px

In [251]:
d=json.loads(json_yr)

In [252]:
d.keys()

dict_keys(['20170802', '20070702'])

In [253]:
json.loads(json_yr)

{'20070702': 107000, '20170802': 108000}

In [254]:
d2 = {parser.parse(k): v for k, v in d.items()}

In [255]:
d2

{datetime.datetime(2007, 7, 2, 0, 0): 107000,
 datetime.datetime(2017, 8, 2, 0, 0): 108000}

In [256]:
sorted(d2.items(),key=lambda x:x,reverse=True)[0]

(datetime.datetime(2017, 8, 2, 0, 0), 108000)

In [259]:
last_sold, sold_px = get_last_sold_from_history(json_yr)

In [261]:
sold_px

108000

In [262]:
prop2 = get_property_soup('https://www.eppraisal.com/home-values/property/2614-springs-dr-charlotte-nc-28226-92151712/')

In [267]:
find_sale_history_from_soup(prop2)

'{"20170717": 350000, "20040430": 243000}'

In [275]:
def get_property_taxes_from_soup(soup):
    """ Pulls last assessed property taxes and appraisal value from bs4
    """
    loops = soup.find_all('div',class_="panel-column panel-column-3")
    for loop in loops:
        if 'Property Taxes:' in loop.text:
            tax = loop.text
            tax_strip = tax[tax.find('Property Taxes: $')+len('Property Taxes: $'):]
            return float(tax_strip.replace(',',''))
            
    

In [276]:
get_property_taxes_from_soup(prop)

1254.06

In [277]:
def get_eppraisal__url_page(pg):
    URL = 'eppraisal.com/recently-sold/nc/charlotte/?pg={}'.format(pg)
    return URL

In [284]:
def scrape_eppraisal_home_page(home_url,city,state):
    """Runs through a list of URLs, scraping home data into a dict
    
    Keyword arguments:
        home_url: [string] a url associated with a property
        city: [string] city of the property being scraped
        state: [string] state of the property being scraped
    
    Returns:
    """
    
    home = get_property_soup(home_url)
    
    h = {}
    response = get_address_from_soup(home,city,state)[0]
    address={}
    address['address_line1'] = str(response['address_info']['address'])
    address['city'] = str(response['address_info']['city'])
    address['zipcode'] = str(response['address_info']['zipcode'])
    address['state'] = str(response['address_info']['state'])
    address['unit'] = str(response['address_info']['unit'])
    address['lat'] = str(response['address_info']['lat'])
    address['lon'] = str(response['address_info']['lng'])
    address['slug'] = str(response['address_info']['slug'])
    h['address'] = address
    
    l = get_bed_bath_sqft_from_soup(home)
    
    sale_price_history = find_sale_history_from_soup(home)
    h['sold_price_history'] = sale_price_history
    
    h['listing_data'] = l
    
    tax = get_property_taxes_from_soup(home)
    h['prop_taxes'] = tax
    last_sold_date, last_sold_px = get_last_sold_from_history(sale_price_history)
    h['year_built'] = get_year_built_from_soup(home)
    h['last_sold_date'] = last_sold_date
    h['last_sold_price'] = last_sold_px
    return h

In [286]:
d = scrape_eppraisal_home_page(urls[4],'Charlotte','NC')

In [287]:
def flatten_dict(dd, separator='_', prefix=''):
    return { prefix + separator + k if prefix else k : v
             for kk, vv in dd.items()
             for k, v in flatten_dict(vv, separator, kk).items()
             } if isinstance(dd, dict) else { prefix : dd }

In [288]:
flat = flatten_dict(d)

In [289]:
flat

{'address_address_line1': '16833 Greenlawn Hills Ct',
 'address_city': 'Charlotte',
 'address_lat': '35.3034',
 'address_lon': '-80.69021',
 'address_slug': '16833-Greenlawn-Hills-Ct-Charlotte-NC-28213',
 'address_state': 'NC',
 'address_unit': 'None',
 'address_zipcode': '28213',
 'last_sold_date': '2017-08-02',
 'last_sold_price': 110000,
 'listing_data_air_conditioning': 'Central',
 'listing_data_basement_area': '',
 'listing_data_building_area_sq_ft': 1094,
 'listing_data_fireplace': '',
 'listing_data_garage_or_park_sqft': '',
 'listing_data_heat_type': 'Yes',
 'listing_data_lot_area_acres': 0.0,
 'listing_data_lot_area_sqft': '-',
 'listing_data_num_bathrooms': 2,
 'listing_data_num_bedrooms': 2,
 'listing_data_roof_type': '',
 'prop_taxes': 12660.96,
 'sold_price_history': '{"20050419": 727500, "20170802": 110000}',
 'year_built': '1971'}

In [280]:
get_bed_bath_sqft_from_soup(prop)

{'air_conditioning': 'Central',
 'basement_area': '',
 'building_area_sq_ft': 1094,
 'fireplace': '',
 'garage_or_park_sqft': '',
 'heat_type': 'Yes',
 'lot_area_acres': 0.0,
 'lot_area_sqft': '-',
 'num_bathrooms': 2,
 'num_bedrooms': 2,
 'roof_type': ''}

In [281]:
get_property_taxes_from_soup(prop)

1254.06

In [290]:
urls[4]

'https://www.eppraisal.com/home-values/property-lookup?m=2137halfordpl28211'

In [292]:
flat['address_slug']

'16833-Greenlawn-Hills-Ct-Charlotte-NC-28213'